# Évaluation du Pipeline : Détection et Identification
Ce notebook a pour but de quantifier les performances de notre système d'analyse de pièces de monnaie. 
L'évaluation se fait en 3 étapes strictes :
1. **Évaluation de la Détection :** Le système trouve-t-il les pièces (Précision, Rappel) ?
2. **Évaluation de l'Identification :** Quand une pièce est trouvée, son métal et sa valeur sont-ils corrects (Matrice de Confusion) ?
3. **Évaluation Globale (End-to-End) :** L'erreur finale sur la somme totale en euros par image.

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm # Pour avoir une belle barre de progression

# Importation de nos propres modules (les fichiers .py doivent être dans le même dossier)
# Assure-toi d'avoir renommé ton fichier de détection en 'coin_detector.py'
from coin_detector import CoinDetection
from coin_identification import CoinIdentification

# Configuration esthétique pour les graphiques
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Initialisation de nos classes
detector = CoinDetection(target_width=800)
# Attention: on désactive le KNN puisqu'il est optionnel et non utilisé dans la méthode finale
identifier = CoinIdentification(use_knn=False) 

# CHEMIN VERS LES IMAGES
IMAGE_DIR = './data/'

### 1. Préparation et Logique de Matching (Association)
Pour évaluer notre modèle, nous devons comparer nos prédictions avec la vérité terrain (`ground_truth.csv`).
**Le problème :** Comment savoir si le cercle détecté en `(x=120, y=300)` correspond bien à la pièce numéro 2 de la vérité terrain ?

**La solution :** Nous utilisons la distance Euclidienne. Si le centre d'une pièce prédite est très proche du centre d'une vraie pièce (ex: moins de 20 pixels d'écart), on considère que c'est un **Vrai Positif (Match)**.

In [4]:
def match_coins(ground_truth, predictions, dist_threshold=30):
    """
    Associe les pièces détectées aux vraies pièces en calculant la distance entre les centres.
    
    Retourne 3 listes :
    - matched: Les paires (Vraie Pièce, Pièce Prédite) -> Vrais Positifs (TP)
    - false_negatives: Vraies pièces non détectées (FN)
    - false_positives: Prédictions qui ne correspondent à aucune vraie pièce (FP)
    """
    matched = []
    unmatched_gt = ground_truth.copy()
    unmatched_pred = predictions.copy()
    
    # Pour chaque vraie pièce, on cherche la prédiction la plus proche
    for gt in ground_truth:
        best_pred = None
        best_dist = float('inf')
        
        for pred in unmatched_pred:
            # Calcul de la distance euclidienne entre les centres (x, y)
            dist = np.hypot(gt['x'] - pred['x'], gt['y'] - pred['y'])
            if dist < best_dist and dist < dist_threshold:
                best_dist = dist
                best_pred = pred
                
        if best_pred is not None:
            matched.append((gt, best_pred))
            unmatched_pred.remove(best_pred)
            unmatched_gt.remove(gt)
            
    return matched, unmatched_gt, unmatched_pred

print("Fonction de matching initialisée. Prêt pour l'évaluation !")

Fonction de matching initialisée. Prêt pour l'évaluation !


### 2. Évaluation de la Détection pure (Localisation)
Ici, on ne s'occupe pas de la couleur ou de l'argent. On veut juste savoir si le système repère bien les ronds de métal.

* **Vrai Positif (TP) :** Pièce détectée au bon endroit.
* **Faux Positif (FP) :** L'algorithme a inventé une pièce (ex: une tache ronde vue comme une pièce).
* **Faux Négatif (FN) :** L'algorithme a raté une pièce (elle est sur la table, mais ignorée).

Nous calculons la **Précision** (La fiabilité de nos détections) et le **Rappel** (Notre capacité à ne rien rater).

In [5]:
# --- SIMULATION POUR LE NOTEBOOK ---
# Dans ton vrai projet, tu chargeras ton ground_truth.csv avec pd.read_csv() 
# et tu feras une boucle sur tes images pour récupérer les prédictions.
# Ici, on simule une image pour la démonstration.

gt_coins = [
    {'id': 1, 'x': 100, 'y': 100, 'color': 'copper', 'denom': '5c'},
    {'id': 2, 'x': 200, 'y': 200, 'color': 'gold', 'denom': '50c'},
    {'id': 3, 'x': 300, 'y': 300, 'color': 'bimetallic', 'denom': '2euro'}
]

pred_coins = [
    {'x': 102, 'y': 98, 'color': 'copper', 'denom': '5c'},     # Match parfait
    {'x': 205, 'y': 195, 'color': 'copper', 'denom': '10c'},   # Match géométrique (mais erreur d'identification)
    {'x': 500, 'y': 500, 'color': 'gold', 'denom': '20c'}      # Faux Positif (Hallucination)
    # La pièce 3 (2euro) a été ratée -> Faux Négatif
]

# 1. On lance le matching
matched, false_negatives, false_positives = match_coins(gt_coins, pred_coins)

TP = len(matched)
FN = len(false_negatives)
FP = len(false_positives)

# 2. Calcul des métriques
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"--- MÉTRIQUES DE DÉTECTION ---")
print(f"Vrais Positifs (Pièces trouvées) : {TP}")
print(f"Faux Positifs (Fausses alertes)  : {FP}")
print(f"Faux Négatifs (Pièces ratées)    : {FN}")
print(f"--------------------------------")
print(f"Précision : {precision*100:.1f}% (Quand je dis 'c'est une pièce', j'ai raison à {precision*100:.1f}%)")
print(f"Rappel    : {recall*100:.1f}% (Sur toutes les pièces existantes, j'en ai trouvé {recall*100:.1f}%)")
print(f"Score F1  : {f1_score*100:.1f}% (Moyenne harmonique)")

--- MÉTRIQUES DE DÉTECTION ---
Vrais Positifs (Pièces trouvées) : 2
Faux Positifs (Fausses alertes)  : 1
Faux Négatifs (Pièces ratées)    : 1
--------------------------------
Précision : 66.7% (Quand je dis 'c'est une pièce', j'ai raison à 66.7%)
Rappel    : 66.7% (Sur toutes les pièces existantes, j'en ai trouvé 66.7%)
Score F1  : 66.7% (Moyenne harmonique)
